Let aussume that we need to make this computation

$\sqrt{(a+b)\times c}$

and we want to compute the standard deviation of the result assuming that a, b and c are normal distributed independent variables. Clearly the problem is quite naive but we want to address is as if we will need a cluster to solve it. 

We can partition the problem in a three conscutive operations
1. A sum: $(a+b)$
2. A multiplication of the result 1 with c: $(a+b)\times c$
3. A sqrt of the result of 2: $\sqrt{(a+b)\times c}$

In [1]:
import tree_maker
from tree_maker import NodeJob

In [33]:
import numpy as np
a=np.random.randn(4)
b=np.random.randn(4)
c=np.random.randn(2)

my_list_original=[]
for ii in c:
    my_list_original+=list(np.sqrt(np.abs((a+b)*ii)))
my_list_original=sorted(my_list_original)

In [34]:
#root
root = NodeJob(name='root', parent=None)
root.path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree'

#first generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child}",
                           parent=node,
                           path = f"{node.path}/{child}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/sum_it',
                           dictionary={'a':float(a[child]), 'b':float(b[child])})
                   for child in range(len(a))]

#second generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child}",
                           parent=node,
                           path = f"{node.path}/{child}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/multiply_it',
                           dictionary={'c':float(c[child])})
                   for child in range(len(c))]
    
#third generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child}",
                           parent=node, 
                           path = f"{node.path}/{child}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/square_root_it',
                           dictionary={'a':float(c[child])})
                           for child in range(1)]

In [35]:
root.print_it()

root
├── 0
│   ├── 0
│   │   ╰── 0
│   ╰── 1
│       ╰── 0
├── 1
│   ├── 0
│   │   ╰── 0
│   ╰── 1
│       ╰── 0
├── 2
│   ├── 0
│   │   ╰── 0
│   ╰── 1
│       ╰── 0
╰── 3
    ├── 0
    │   ╰── 0
    ╰── 1
        ╰── 0


In [36]:
#import numpy as np
#root.path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree'

# sum
#for ii, node in enumerate(root.generation(1)): 
#    node.path = node.parent.path + '/' + node.name
#    node.template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/sum_it'
#    node.dictionary={'a':float(a[ii]), 'b': float(b[ii])}
    
# product
#for ii, node in enumerate(root.generation(2)): 
#    node.path = node.parent.path + '/' + node.name
#    node.template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/multiply_it'

# sqrt
#for node in root.generation(3): 
#    node.path = node.parent.path + '/' + node.name
#    node.template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/templates/square_root_it'
#    node.dictionary={'a':float(0)}

In [37]:
# save the tree
root.to_yaml()

In [38]:
# Load the tree from a yaml
root=tree_maker.tree_from_yaml('./node.yaml')
root.print_it()

root
├── 0
│   ├── 0
│   │   ╰── 0
│   ╰── 1
│       ╰── 0
├── 1
│   ├── 0
│   │   ╰── 0
│   ╰── 1
│       ╰── 0
├── 2
│   ├── 0
│   │   ╰── 0
│   ╰── 1
│       ╰── 0
╰── 3
    ├── 0
    │   ╰── 0
    ╰── 1
        ╰── 0


In [39]:
root.children[0].dictionary

{'a': -1.0938419614202788, 'b': -1.5964716114815558}

In [40]:
# STEP 1 cloning
root.rm_children()
[x.clone_children() for x in root.generation(0)]
[x.clone_children() for x in root.generation(1)]
[x.clone_children() for x in root.generation(2)];

In [41]:
for node in root.generation(1):
    node.mutate()
    node.run()

In [42]:
node=root.generation(2)[0]
tree_maker.from_yaml(node.parent.path+'/output.yaml')

ordereddict([('result', -2.6903135729018346)])

In [43]:
for node in root.generation(2):
    parent_output = tree_maker.from_yaml(node.parent.path+'/output.yaml')
    node.dictionary['sum_a_b']=parent_output['result']
    node.mutate()
    node.run()

In [44]:
for node in root.generation(3):
    parent_output = tree_maker.from_yaml(node.parent.path+'/output.yaml')
    node.dictionary['a']=parent_output['result']
    node.mutate()
    node.run()

In [45]:
my_list=[]
for node in root.generation(3):
    output = tree_maker.from_yaml(node.path+'/output.yaml')
    my_list.append(output['result'])

In [47]:
sorted(my_list)

[0.12662002366830288,
 0.223856033239695,
 0.22765041897551555,
 0.4024712543942659,
 0.6746578231021625,
 0.9135330843974122,
 1.1927515072135049,
 1.6150675586836207]

In [48]:
my_list_original

[0.12662002366830288,
 0.223856033239695,
 0.22765041897551555,
 0.4024712543942659,
 0.6746578231021625,
 0.9135330843974122,
 1.1927515072135049,
 1.6150675586836207]